In [1]:
import numpy as np
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import os
from ProjectF import MLAData,classification, Object,storing,MLADataBin
import random

## Loading in data
PlateDir = os.path.normpath("D:\Data\Plate_Name_Reduced.txt")
with open(PlateDir) as f:
    Spectra_Files = f.read().splitlines()
print('Yes')  
PLATEIDs = []
BinInfos = []
Flux = []
MJDs = []
log_wavst=[]
ORMASK=[]
ANDMASK=[]
INVAR=[]

TrainingDir = os.path.normpath("D:\Data")
slash =  os.path.normpath("\\")
TrainingFolder =  os.path.normpath("\Training")
slash =  os.path.normpath("\\")
for spectrum in Spectra_Files:
    plate_ = fits.open( TrainingDir +TrainingFolder+slash+ spectrum ,memmap=True)
    Bin_info_ = plate_[5].data
    Flux_ = plate_[0].data
    primhdu_ = plate_[0]
    PLATEIDs.append(primhdu_.header['PLATEID'])
    ORMASK.append( plate_[3].data)
    ANDMASK.append( plate_[2].data)
    INVAR.append( plate_[1].data)
    log_wavst.append(primhdu_.header['COEFF0'])
    MJDs.append(primhdu_.header['MJD'])
    BinInfos.append(Bin_info_)
    Flux.append(Flux_)
    
list = fits.open(TrainingDir+slash+'Superset_DR12Q.fits',memmap=True)#opening file
print(len(log_wavst))
supers=list[1].data # storing  BINTABLE extension data

Full_Data = storing(PLATEIDs,supers)






Yes
10


In [2]:
X,Y,Train_z, Train_mag,wavst = MLAData(Full_Data,BinInfos,Flux, log_wavst)

In [3]:
i=0
#while i <len(PLATEIDs):
C_Plate = PLATEIDs[i]
a1 = X[i] 
a2 = np.array(Y[i])
col1 = fits.Column(name='Bin_Flux', format='PJ()', array=np.array(a1,dtype=np.object))
col2 = fits.Column(name='Class', format='I', array=a2)
cols = fits.ColDefs([col1, col2])
tbhdu = fits.BinTableHDU.from_columns(cols)
print(tbhdu.data)
tbhdu.writeto(TrainingDir+slash+np.str(C_Plate)+'.fits')
i=i+1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [ ]:
BIN_Size=3
X_bin,Y_bin,Train_z_bin, Train_mag_bin,wavst_bin = MLADataBin(Full_Data,BinInfos,Flux,log_wavst, ANDMASK, INVAR, BIN_Size)
    

In [ ]:
i=0
while i <len(PLATEIDs):
    C_Plate = PLATEIDs[i]
    a1 = X_bin[i] 
    a2 = np.array(Y_bin[i])
    col1 = fits.Column(name='Bin_Flux', format='PJ()', array=np.array(a1,dtype=np.object))
    col2 = fits.Column(name='Class', format='I', array=a2)
    cols = fits.ColDefs([col1, col2])
    tbhdu = fits.BinTableHDU.from_columns(cols)
    tbhdu.writeto(TrainingDir+slash+"rebinned"+np.str(C_Plate)+'.fits')
    i=i+1